<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:
### Hidden Layer:
### Output Layer:
### Neuron:
### Weight:
### Activation Function:
### Node Map:
### Perceptron:


## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

#### Your Answer Here

## Write your own perceptron code that can correctly classify (99.0% accuracy) a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [1]:
import pandas as pd
data = { 'x1': [0,1,0,1],
         'x2': [0,0,1,1],
         'y':  [1,1,1,0]
       }

df = pd.DataFrame.from_dict(data).astype('int')
df

,x1,x2,y
0,0,0,1
1,1,0,1
2,0,1,1
3,1,1,0


In [87]:
import numpy as np

target = ['y']
features = ['x1', 'x2']

X_train = df[features]
X_train['bias'] = 1
y_train = df[target]

**Sigmoid activation function and its derivative for updating weights**

In [88]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1 - sx)

In [121]:
# Initialize random weights for each of our two inputs

weights = 2 * np.random.random((X_train.shape[1],1)) - 1
print('weights before training')
print(weights)
print()

bias = 5
print('bias:', bias)
print()
# each iteration is an epoch

for iteration in range(10):
    # weighted sum
    weighted_sum = np.dot(X_train, weights)
    
    # activate
    activated_output = sigmoid(weighted_sum + bias)
    
    # calculate error
    error = y_train - activated_output
    
    # calculate adjustments
    adjustments = error * sigmoid_derivative(activated_output)
    
    # update the weights
    weights += np.dot(X_train.T, adjustments)
    
print('weights after training')
print(weights)
print()
print('outputs after training')
print(activated_output)
print()


weights before training
[[-0.93234977]
 [ 0.8264471 ]
 [-0.34076717]]

bias: 5

weights after training
[[-2.40608687]
 [-0.85887569]
 [-1.71558831]]

outputs after training
[[0.96560278]
 [0.7310689 ]
 [0.9303533 ]
 [0.5639982 ]]



In [122]:
from sklearn.metrics import accuracy_score

y_pred = np.round(activated_output).astype(int)
print('predictions:')
print(y_pred)
print()
print('accuracy:', accuracy_score(y_train, y_pred))

predictions:
[[1]
 [1]
 [1]
 [1]]

accuracy: 0.75


## Implement your own Perceptron Class and use it to classify a binary dataset: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [55]:
diabetes = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [56]:
diabetes.shape

(768, 9)

In [57]:
diabetes.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

Although neural networks can handle non-normalized data, scaling or normalizing your data will improve your neural network's learning speed. Try to apply the sklearn `MinMaxScaler` or `Normalizer` to your diabetes dataset. 

In [58]:
from sklearn.preprocessing import MinMaxScaler, Normalizer

target = 'Outcome'
features = diabetes.drop(columns=target).columns

X = diabetes[features]
y = diabetes[target]

In [115]:
w = 2 * np.random.random((X.shape[1], 1)) - 1
ws = np.dot(X, w)
ao = sigmoid(ws + 123)
e=[y] - ao
e.shape

(768, 768)

In [137]:
import numpy as np

class Perceptron:
    
    def __init__(self, niter=10, bias=1):
        self.niter = niter
        self.bias = bias
    
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def __sigmoid_derivative(self, x):
        sx = self.__sigmoid(x)
        return sx * (1 - sx)

    def fit(self, X, y):
        """Fit training data
        X : Training vectors, X.shape : [#samples, #features]
        y : Target values, y.shape : [#samples]
        """

        # Randomly Initialize Weights
        X['bias'] = 1
        weights = 2 * np.random.random((X.shape[1], 1)) - 1

        for i in range(self.niter):
            # Weighted sum of inputs / weights
            weighted_sum = np.dot(X, weights)

            # Activate!
            activated_output = self.__sigmoid(weighted_sum)

            # Cac error
            error = [y] - activated_output
            adjustments = error * self.__sigmoid_derivative(activated_output)

            # Update the Weights
            weights += np.dot(X.T, adjustments)

        self.weights_ = weights

        return self


    def predict(self, X):
        """Return class label after unit step"""
    
        # weighted sum of inputs and weights
        weighted_sum = np.dot(X, self.weights_)
        
        # activate
        activated_output = self.__sigmoid(weighted_sum + self.bias)
        
        # convert to discrete predictions
        pred = np.round(activated_output).astype(int)
        
        return pred

In [138]:
perceptron = Perceptron().fit(X, y)

ValueError: non-broadcastable output operand with shape (9,1) doesn't match the broadcast shape (9,768)

## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?